# Risk Model Pipeline - Data Dictionary Example

This notebook demonstrates how to use the Risk Model Pipeline with:
- Data dictionary support for variable descriptions
- Proper model report extraction
- DataFrame-based calibration
- WOE reports with monotonic ordering

In [ ]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime, timedelta
import sys
import os

# Add parent directory to path
sys.path.append('..')

# Import pipeline components
from src.risk_pipeline.pipeline16 import RiskModelPipeline, Config
from src.risk_pipeline.utils.pipeline_runner import run_pipeline_from_dataframe

warnings.filterwarnings('ignore')

print("Pipeline modules imported successfully!")

## 1. Generate Sample Data

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate sample data
n_samples = 5000

# Create base date range
start_date = datetime(2023, 1, 1)
dates = [start_date + timedelta(days=i) for i in range(n_samples)]

# Generate features
df = pd.DataFrame({
    'app_id': [f'APP_{i:06d}' for i in range(n_samples)],
    'app_dt': dates,
    'target': np.random.binomial(1, 0.2, n_samples),  # 20% default rate
    
    # Numeric features
    'age': np.random.randint(18, 70, n_samples),
    'income': np.random.lognormal(10, 1, n_samples),  # Log-normal income distribution
    'credit_score': np.random.normal(650, 100, n_samples).clip(300, 850),
    'loan_amount': np.random.exponential(50000, n_samples),
    'employment_years': np.random.exponential(5, n_samples).clip(0, 40),
    
    # Categorical features
    'education': np.random.choice(['High School', 'Bachelor', 'Master', 'PhD'], n_samples, p=[0.3, 0.4, 0.25, 0.05]),
    'employment_type': np.random.choice(['Salaried', 'Self-Employed', 'Retired', 'Student'], n_samples, p=[0.6, 0.25, 0.1, 0.05]),
    'marital_status': np.random.choice(['Single', 'Married', 'Divorced', 'Widowed'], n_samples, p=[0.3, 0.5, 0.15, 0.05]),
    'property_type': np.random.choice(['Own', 'Rent', 'Family', 'Other'], n_samples, p=[0.4, 0.35, 0.2, 0.05]),
    'city_tier': np.random.choice([1, 2, 3, 4], n_samples, p=[0.3, 0.35, 0.25, 0.1])
})

# Add some missing values
missing_indices = np.random.choice(df.index, size=int(0.05 * len(df)), replace=False)
df.loc[missing_indices, 'employment_years'] = np.nan

missing_indices = np.random.choice(df.index, size=int(0.03 * len(df)), replace=False)
df.loc[missing_indices, 'marital_status'] = np.nan

print(f"Dataset shape: {df.shape}")
print(f"Target distribution:\n{df['target'].value_counts(normalize=True)}")
print(f"\nFeatures: {list(df.columns[3:])}")
print(f"\nDate range: {df['app_dt'].min()} to {df['app_dt'].max()}")

## 2. Create Data Dictionary (Veri Sözlüğü)

In [ ]:
# Create data dictionary with Turkish descriptions
data_dictionary = pd.DataFrame({
    'alan_adi': [
        'age', 'income', 'credit_score', 'loan_amount', 'employment_years',
        'education', 'employment_type', 'marital_status', 'property_type', 'city_tier'
    ],
    'alan_aciklamasi': [
        'Müşteri yaşı (yıl)',
        'Aylık gelir tutarı (TL)',
        'Kredi skoru (300-850 arası)',
        'Talep edilen kredi tutarı (TL)',
        'Toplam çalışma süresi (yıl)',
        'Eğitim durumu',
        'İstihdam türü',
        'Medeni durum',
        'Konut sahiplik durumu',
        'Şehir kategorisi (1-4)'
    ]
})

print("Data Dictionary (Veri Sözlüğü):")
print(data_dictionary.to_string(index=False))

## 3. Run Pipeline with Data Dictionary

In [ ]:
# Configure and run pipeline
cfg = Config(
    id_col='app_id',
    time_col='app_dt',
    target_col='target',
    use_test_split=True,
    test_size_row_frac=0.2,
    oot_window_months=3,
    output_folder='outputs',
    output_excel_path='model_report_with_dict.xlsx',
    data_dictionary_df=data_dictionary,  # Pass data dictionary
    cv_folds=3,
    random_state=42,
    # Quick run for demo
    hpo_timeout=60,
    hpo_n_trials=10,
    rare_threshold=0.02,
    psi_threshold=0.20
)

print("Starting pipeline run with data dictionary...")
print("="*60)

# Create and run pipeline
pipeline = RiskModelPipeline(cfg)
pipeline.run(df)

# Note: pipeline.run() returns the pipeline object itself for method chaining
# NOT a dictionary - this was the previous error

print("\n" + "="*60)
print("Pipeline completed successfully!")

## 4. Extract Model Results (No .get() Error!)

In [ ]:
# Correct way to access pipeline results - directly from pipeline attributes
print("Model Results:")
print("="*60)

# Access run ID from config
print(f"Run ID: {pipeline.cfg.run_id}")

# Access best model
print(f"Best Model: {pipeline.best_model_name_}")

# Access final features
print(f"Number of Final Features: {len(pipeline.final_vars_)}")
print(f"Final Features: {pipeline.final_vars_[:10]}...")  # Show first 10

# Access model performance
if pipeline.models_summary_ is not None:
    print("\nModel Performance Summary:")
    print(pipeline.models_summary_[['model', 'auc_oot', 'gini_oot']].to_string())

## 5. View Best Model Variables with Descriptions

In [ ]:
# Check best model variables with descriptions from data dictionary
if pipeline.best_model_vars_df_ is not None:
    print("Best Model Variables with Descriptions:")
    print("="*80)
    
    # Display variables with their descriptions
    display_cols = ['variable', 'description', 'coef_or_importance', 'variable_group']
    available_cols = [col for col in display_cols if col in pipeline.best_model_vars_df_.columns]
    
    print(pipeline.best_model_vars_df_[available_cols].head(15).to_string())
    
    # Check if descriptions were loaded
    has_descriptions = pipeline.best_model_vars_df_['description'].notna().any()
    print(f"\nDescriptions loaded: {'Yes ✓' if has_descriptions else 'No ✗'}")

## 6. View WOE Report with Monotonic Ordering

In [ ]:
# Check WOE report structure
if pipeline.best_model_woe_df_ is not None:
    print("WOE Report Structure:")
    print("="*80)
    
    # Show available columns
    print(f"Columns: {list(pipeline.best_model_woe_df_.columns)}")
    
    # Get first variable for demonstration
    first_var = pipeline.best_model_woe_df_['variable'].iloc[0]
    var_woe = pipeline.best_model_woe_df_[pipeline.best_model_woe_df_['variable'] == first_var]
    
    print(f"\nWOE bins for '{first_var}' (sorted by event_rate - monotonic):")
    print("-"*80)
    
    # Show description if available
    if 'variable_description' in var_woe.columns:
        desc = var_woe['variable_description'].iloc[0]
        if desc:
            print(f"Description: {desc}")
    
    # Display WOE bins
    display_cols = ['group', 'bin_from', 'bin_to', 'count', 'event_rate', 'woe']
    available_cols = [col for col in display_cols if col in var_woe.columns]
    
    print("\n" + var_woe[available_cols].to_string())
    
    # Check monotonicity
    event_rates = var_woe['event_rate'].values
    is_monotonic = all(event_rates[i] <= event_rates[i+1] for i in range(len(event_rates)-1))
    print(f"\nMonotonic ordering: {'Yes ✓' if is_monotonic else 'No ✗'}")

## 7. Alternative: Using Helper Function

In [ ]:
# You can also use the helper function for simpler usage
from src.risk_pipeline.utils.pipeline_runner import run_pipeline_from_dataframe

# Run with helper function
results = run_pipeline_from_dataframe(
    df=df,
    data_dictionary_df=data_dictionary,  # Pass data dictionary
    output_folder='outputs_helper',
    output_excel='model_report_helper.xlsx',
    use_test_split=True,
    oot_months=3,
    hpo_timeout=30,
    hpo_n_trials=5
)

# With helper function, results is a dictionary
print("Results from helper function:")
print(f"Best model: {results['best_model']}")
print(f"Number of final features: {len(results['final_features'])}")
print(f"Run ID: {results['run_id']}")
print(f"Output folder: {results['output_folder']}")

## 8. Load and View Excel Report

In [ ]:
# Load the Excel report to verify data dictionary integration
import os

excel_path = os.path.join(pipeline.cfg.output_folder, pipeline.cfg.output_excel_path)

if os.path.exists(excel_path):
    # Load Excel file
    excel_file = pd.ExcelFile(excel_path)
    
    print(f"Excel report created: {excel_path}")
    print(f"\nAvailable sheets ({len(excel_file.sheet_names)}):")
    for i, sheet in enumerate(excel_file.sheet_names, 1):
        print(f"{i:2}. {sheet}")
    
    # Check best_model_vars sheet
    if 'best_model_vars_df' in excel_file.sheet_names:
        best_vars_df = pd.read_excel(excel_path, sheet_name='best_model_vars_df')
        print("\nBest Model Variables from Excel:")
        print(best_vars_df[['variable', 'description', 'coef_or_importance']].head(5))
    
    # Check WOE report sheet
    if 'best_model_woe_df' in excel_file.sheet_names:
        woe_df = pd.read_excel(excel_path, sheet_name='best_model_woe_df')
        print("\nWOE Report from Excel (first variable):")
        first_var = woe_df['variable'].iloc[0]
        print(woe_df[woe_df['variable'] == first_var][['group', 'event_rate', 'woe']].head())
else:
    print(f"Excel report not found at {excel_path}")

## 9. Example: Calibration with DataFrame

In [ ]:
# Generate calibration data
n_cal = 1000
cal_df = pd.DataFrame({
    'app_id': [f'CAL_{i:06d}' for i in range(n_cal)],
    'app_dt': pd.date_range('2023-12-01', periods=n_cal, freq='H'),
    'target': np.random.binomial(1, 0.25, n_cal),  # Slightly different default rate
    
    # Same features as training
    'age': np.random.randint(18, 70, n_cal),
    'income': np.random.lognormal(10, 1, n_cal),
    'credit_score': np.random.normal(650, 100, n_cal).clip(300, 850),
    'loan_amount': np.random.exponential(50000, n_cal),
    'employment_years': np.random.exponential(5, n_cal).clip(0, 40),
    'education': np.random.choice(['High School', 'Bachelor', 'Master', 'PhD'], n_cal),
    'employment_type': np.random.choice(['Salaried', 'Self-Employed', 'Retired', 'Student'], n_cal),
    'marital_status': np.random.choice(['Single', 'Married', 'Divorced', 'Widowed'], n_cal),
    'property_type': np.random.choice(['Own', 'Rent', 'Family', 'Other'], n_cal),
    'city_tier': np.random.choice([1, 2, 3, 4], n_cal)
})

print(f"Calibration data shape: {cal_df.shape}")
print(f"Calibration target rate: {cal_df['target'].mean():.3f}")

# Run pipeline with calibration DataFrame
cfg_with_cal = Config(
    id_col='app_id',
    time_col='app_dt',
    target_col='target',
    use_test_split=True,
    oot_window_months=2,
    output_folder='outputs_calibrated',
    output_excel_path='model_report_calibrated.xlsx',
    data_dictionary_df=data_dictionary,  # Data dictionary
    calibration_df=cal_df,  # Calibration DataFrame
    cv_folds=3,
    hpo_timeout=30,
    hpo_n_trials=5
)

print("\nRunning pipeline with calibration...")
pipeline_cal = RiskModelPipeline(cfg_with_cal)
pipeline_cal.run(df)

print(f"\nCalibration applied: {pipeline_cal.calibrator_ is not None}")
print(f"Best model: {pipeline_cal.best_model_name_}")

## Summary

This notebook demonstrated:

1. **Data Dictionary Support**: Variables now have Turkish descriptions in reports
2. **Correct Result Access**: No more `.get()` errors - access attributes directly from pipeline
3. **WOE Monotonic Ordering**: Bins sorted by default rate for better risk understanding
4. **Enhanced WOE Report**: Separate columns for bin ranges (bin_from, bin_to)
5. **DataFrame Calibration**: Direct DataFrame support without requiring CSV files

The Excel report (`model_report_with_dict.xlsx`) contains all results with variable descriptions!